In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 第一章 预备知识

### Ex1：利用列表推导式写矩阵乘法

求解后进行验证

In [12]:
M1 = np.random.rand(2,3)
M2 = np.random.rand(3,4)
res = [[sum([M1[i][k] * M2[k][j] for k in range(M1.shape[1])]) for j in range(M2.shape[1])] for i in range(M1.shape[0])]
print(((M1@M2 - res) < 1e-15).all())

True


验证正确，矩阵乘法

### Ex2：更新矩阵

In [13]:
A = np.arange(1,10).reshape(3,-1)
B = A*(1/A).sum(1).reshape(-1,1)
print(A)
print(B)

[[1 2 3]
 [4 5 6]
 [7 8 9]]
[[1.83333333 3.66666667 5.5       ]
 [2.46666667 3.08333333 3.7       ]
 [2.65277778 3.03174603 3.41071429]]


### Ex3：卡方统计量

In [14]:
np.random.seed(0)
A = np.random.randint(10, 20, (8, 5))
B = A.sum(0)*A.sum(1).reshape(-1, 1)/A.sum()
res = ((A-B)**2/B).sum()
print(res)

11.842696601945802


### Ex4：改进矩阵计算的性能

原方法：

In [21]:
np.random.seed(0)
m, n, p = 100, 80, 50
B = np.random.randint(0, 2, (m, p))
U = np.random.randint(0, 2, (p, n))
Z = np.random.randint(0, 2, (m, n))
def solution(B=B, U=U, Z=Z):
    L_res = []
    for i in range(m):
        for j in range(n):
            norm_value = ((B[i]-U[:,j])**2).sum()
            L_res.append(norm_value*Z[i][j])
    return sum(L_res)
print(solution(B, U, Z))

100566


改进方法：

令$Y_{ij} = \|B_i-U_j\|_2^2$，则$\displaystyle R=\sum_{i=1}^m\sum_{j=1}^n Y_{ij}Z_{ij}$，这在`Numpy`中可以用逐元素的乘法后求和实现，因此问题转化为了如何构造`Y`矩阵。

$$
\begin{split}Y_{ij} &= \|B_i-U_j\|_2^2\\
&=\sum_{k=1}^p(B_{ik}-U_{kj})^2\\
&=\sum_{k=1}^p B_{ik}^2+\sum_{k=1}^p U_{kj}^2-2\sum_{k=1}^p B_{ik}U_{kj}\\\end{split}
$$

从上式可以看出，第一第二项分别为$B$的行平方和与$U$的列平方和，第三项是两倍的内积。因此，$Y$矩阵可以写为三个部分，第一个部分是$m×n$的全$1$矩阵每行乘以$B$对应行的行平方和，第二个部分是相同大小的全$1$矩阵每列乘以$U$对应列的列平方和，第三个部分恰为$B$矩阵与$U$矩阵乘积的两倍。从而结果如下：

In [20]:
print((((B**2).sum(1).reshape(-1,1) + (U**2).sum(0) - 2*B@U)*Z).sum())

100566


对比它们的性能：

In [17]:
%timeit -n 30 solution(B, U, Z)

55.4 ms ± 2.5 ms per loop (mean ± std. dev. of 7 runs, 30 loops each)


In [18]:
%timeit -n 30 ((np.ones((m,n))*(B**2).sum(1).reshape(-1,1) + np.ones((m,n))*(U**2).sum(0) - 2*B@U)*Z).sum()

841 µs ± 90.2 µs per loop (mean ± std. dev. of 7 runs, 30 loops each)


### Ex5：连续整数的最大长度

In [22]:
f = lambda x:np.diff(np.nonzero(np.r_[1,np.diff(x)!=1,1])).max()
print(f([1,2,5,6,7]))
print(f([3,2,1,2,3,4,6]))

3
4
